# Assignment 3

本次作业的目标是通过手动实现神经网络的前向输出和反向梯度计算及参数更新过程，加深对神经网络前反向传播的理解。我们以多层MLP为例展开，第一部分将构建模型并检验手动计算的正确性，第二部分将以一个实例来观察模型的训练过程。

本次作业需要安装pytorch库（深度学习框架）与matplotlib库（绘图工具），确保已经安装完毕后，可运行如下导入语句开始本次作业。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import copy
import torch.optim as optim

## 一、建构模型、实现手动求导并检验

第一步，我们将构造一个多层感知机（MLP）模型，并定义其前向（forward）和反向（backward）过程。

一个L层MLP的模型结构是 {Affine -> Activation} x (L - 1) -> Affine，这里为了简便省略了layer norm和dropout层。其中Affine是仿射层，Activation是激活函数，这里我们以Sigmoid作为代表。

一个$L$层的多层感知机模型可以表示为如下形式：
$$
    x\xrightarrow{W_1}h_1\xrightarrow{\sigma(\cdot)}z_1\xrightarrow{W_2}h_2\xrightarrow{\sigma(\cdot)}z_2\cdots\xrightarrow{W_L}h_L\xrightarrow{f(\cdot)}l,
$$
其中$x\in\mathbb{R}^{d_0},\,h_i\in\mathbb{R}^{d_i},\,z_k\in\mathbb{R}^{d_i},\,W_i\in\mathbb{R}^{d_{i-1},d_i},\,\sigma(\cdot)$为Sigmoid激活函数，$f(\cdot):\mathbb{R}^{d_L}\rightarrow\mathbb{R}$为可求导的损失函数。

下面我们开始构建模型。在pytorch中，我们通常通过定义`torch.nn.Module`的子类来建构模型。

父类`torch.nn.Module`主要的特性是其使用`.forward`方法重载了魔法方法`.__call`，这表示，如果直接将实例名称像函数一样调用，实现的就是其`forward`方法。

`nn.Linear`为pytorch的一个内置的`Module`子类，需要声明输入维度和输出维度，它内含了两个可学习参数——权重矩阵`weight`与偏置`bias`，并默认其`requires_grad=True`，表示其需要求梯度，即它们是可学习的而非固定的参数，这两个参数在未被修改前是随机初始化的，取决于计算机的内存残余数据。

在下面的代码中，请根据参考的初始化网络参数函数，补充`forward`和`backward`函数。

`forward`函数是网络的前向传播，输出模型的预测，为了实现手动求导，我们在模型初始化时提供了用于缓存激活值的字典`activations`（激活值），它并不仅代表激活函数后的输出，而是所有计算图中可能需要的中间计算值，你可以在定义`forward`函数的同时缓存你需要的中间值。

`backward`函数是我们额外定义的网络的反向传播方法，用于实现手动计算梯度。它以模型输出$h_L$的梯度为输入，同时根据模型参数、前向传播时缓存的激活值、已计算出的每一层激活值的梯度等计算每一层参数的梯度。

在书写过程中，你需要注意：pytorch中一维向量默认为行向量，你可以使用`@`, `torch.matmul`, `torch.einsum`等多种形式实现矩阵乘积，但一定要保证维度对应正确。

In [2]:
class CustomMLP(nn.Module):
    def __init__(self, input_dim: int, hidden_dims: list, output_dim: int):
        super().__init__()
        self.layers = nn.ModuleList()
        prev_dim = input_dim

        for hidden_dim in hidden_dims:
            self.layers.append(nn.Linear(in_features=prev_dim, out_features=hidden_dim))
            prev_dim = hidden_dim

        self.output_layer = nn.Linear(in_features=prev_dim, out_features=output_dim)

        # 缓存激活值的字典
        self.activations = {}

    def forward(self, x):
        # 缓存输入值，使用detach()方法使其从计算图中分离。
        self.activations["input"] = x.detach()

        ####################################################################################################
        # TODO: 前向传播，根据网络结构计算：{Affine -> Sigmoid} x (L - 1) -> Affine，将激活值保存在self.activations中
        # 例如第一层，self.activations["fc0"]和self.activations["sigmoid0"]分别保存线性层和激活函数的输出
        ####################################################################################################

        # 隐藏层
        for i, layer in enumerate(self.layers):
            x = layer(x)
            self.activations[f"fc{i}"] = x.detach()
            x = F.sigmoid(x)
            self.activations[f"sigmoid{i}"] = x.detach()

        # 输出层
        x = self.output_layer(x)
        self.activations["output"] = x.detach()

        return x

    def backward(self, grad_output):

        ####################################################################################################
        # TODO: 反向传播，根据前向传播和缓存激活值计算梯度，将其保存在layer.weight.grad和layer.bias.grad中
        # 例如第一层，self.layers[0].weight.grad和self.layers[0].bias.grad分别保存权重和偏置的梯度
        ####################################################################################################

        # 输出层梯度，请将 None 修改为你计算的结果。
        grad = grad_output

        output_input = self.activations[f"sigmoid{len(self.layers)-1}"]
        self.output_layer.weight.grad = grad.T @ output_input
        self.output_layer.bias.grad = grad.sum(dim=0)

        for i in range(len(self.layers) - 1, -1, -1):
            sigmoid_output = self.activations[f"sigmoid{i}"]
            grad = grad @ self.output_layer.weight if i == len(self.layers) - 1 else grad @ self.layers[i + 1].weight
            sigmoid_grad = sigmoid_output * (1 - sigmoid_output)
            grad = grad * sigmoid_grad

            if i > 0:
                layer_input = self.activations[f"sigmoid{i-1}"]
            else:
                layer_input = self.activations["input"]
            
            self.layers[i].weight.grad = grad.T @ layer_input
            self.layers[i].bias.grad = grad.sum(dim=0)
        
        return grad

下面我们通过与自动梯度计算方法比较来检查上述实现的前向和反向过程是否正确，这里使用均方误差（MSE），其计算公式为：
$$
    l(y_{\mathrm{pred}},y_{\mathrm{true}})=\frac{1}{d}\|y_{\mathrm{pred}}-y_{\mathrm{true}}\|_2^2,
$$
其中 $d$ 为 $y_{\mathrm{true}}$ 的维数。

In [3]:
# 模型和数据
mlp = CustomMLP(input_dim=32, hidden_dims=[16, 4], output_dim=2)

# 使用copy方法确保模型初始参数一致
mlp_manual = CustomMLP(input_dim=32, hidden_dims=[16, 4], output_dim=2)
mlp_autograd = copy.deepcopy(mlp_manual)

# 生成随机数据
x = torch.randn(32)
y_true = torch.randn(2) 

# 前向传播
y_pred_manual = mlp_manual(x)
y_pred_autograd = mlp_autograd(x)

# 计算均方误差
loss_manual = F.mse_loss(y_pred_manual, y_true)
loss_autograd = F.mse_loss(y_pred_autograd, y_true)

print("Output Error:", torch.norm(y_pred_manual - y_pred_autograd).item())

Output Error: 0.0


对于手动定义的backward方法，我们先要计算损失对于模型输出的梯度$\frac{\partial l}{\partial y_{\mathrm{pred}}}$，然后将该梯度传入`backward`进行反向传播；对于自动计算，使用loss.backward()方法。

In [4]:
# 手动计算梯度
####################################################################################################
# TODO: 计算出模型输出层的梯度 dloss/dy_pred

####################################################################################################

output_dim = y_pred_manual.shape[0]
grad_output = 2.0 / output_dim * (y_pred_manual - y_true)

# 其余层的梯度使用先前定义的bachward方法计算
mlp_manual.backward(grad_output)

# 使用autograd计算梯度
loss_autograd.backward()

/tmp/ipykernel_245902/3803536256.py:49: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3697.)
  self.output_layer.weight.grad = grad.T @ output_input


RuntimeError: inconsistent tensor size, expected tensor [2] and src [4] to have the same number of elements, but got 2 and 4 elements respectively

首先检查梯度形状：例如，对于第一层，其权重梯度 fc0.weight.grad 的形状应为（hidden_dims[0], input_dim),偏置梯度 fc0.bias.grad 的形状应为（hidden_dims[0]）。

In [ ]:
# 打印各层梯度的形状
for i, layer in enumerate(mlp_manual.layers):
    print(f"fc{i}.weight.grad shape: {layer.weight.grad.shape}")
    print(f"fc{i}.bias.grad shape: {layer.bias.grad.shape}")
print(f"output_layer.weight.grad shape: {mlp_manual.output_layer.weight.grad.shape}")
print(f"output_layer.bias.grad shape: {mlp_manual.output_layer.bias.grad.shape}")

观察各层权重和偏置的梯度差异。如果你实现的backward函数正确，所有的误差不应超过1e-6。

In [ ]:
for i, (layer_manual, layer_auto) in enumerate(zip(mlp_manual.layers, mlp_autograd.layers)):
    print(f"fc{i}.weight.grad error: {torch.norm(layer_manual.weight.grad - layer_auto.weight.grad).item()}")
    print(f"fc{i}.bias.grad error: {torch.norm(layer_manual.bias.grad - layer_auto.bias.grad).item()}")

print(f"output_layer.weight.grad error: {torch.norm(mlp_manual.output_layer.weight.grad - mlp_autograd.output_layer.weight.grad).item()}")
print(f"output_layer.bias.grad error: {torch.norm(mlp_manual.output_layer.bias.grad - mlp_autograd.output_layer.bias.grad).item()}")


## 二、实例：MNIST数据集分类

MNIST数据集是一个广泛使用的机器学习数据集，主要用于手写数字识别任务。它包含了28x28像素的灰度图像，这些图像涵盖了从0到9的十个数字类别，每个类别包含了大量的手写样本。整个数据集分为训练集和测试集两部分，其中训练集包含60,000个样本，用于模型训练；测试集包含10,000个样本，用于评估模型性能。MNIST数据集由于其相对简单且易于理解的特点，常被用作算法原型设计和初步验证的基准数据集。

请你运行该部分的代码，同时学习pytorch深度学习实验的基本模式。

### 导入数据集

In [2]:
import torch
print("PyTorch 版本:", torch.__version__)

PyTorch 版本: 2.5.1


In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 将图片展平，转换为tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# 使用DataLoader加载数据
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


### 训练

In [ ]:
input_dim = 784
hidden_dims = [256]
output_dim = 10

model = CustomMLP(input_dim, hidden_dims, output_dim)

注意和上面不同的是，对于分类任务我们采用了交叉熵损失（cross entropy）。

In [ ]:
# 训练参数
num_epochs = 25
learning_rate = 0.01
loss_history = []

# 定义优化器
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_loss = 0
    for images, labels in train_loader:
        # 前向传播
        outputs = model.forward(images)

        # 计算交叉熵损失
        loss = F.cross_entropy(outputs, labels)
        total_loss += loss.item()

        # 反向传播
        loss.backward()

        # 更新参数
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(train_loader)
    loss_history.append(avg_loss)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")


In [ ]:
# 绘制损失函数下降曲线
plt.figure(figsize=(6, 4))
plt.plot(range(1, num_epochs + 1), loss_history, linestyle='-')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.grid(True)
plt.show()

In [ ]:
def test_model(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model.forward(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


test_model(model, test_loader)